In [1]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from finetunning import squad_tokenizer_prompt, squad_tokenizer_answer

In [2]:
data = load_dataset("squad").remove_columns(["id", "title"])

train, validation = data["train"], data["validation"]

Reusing dataset squad (/home/rocabrera/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
MODEL_NAME = "t5-base"

In [4]:
# One can use T5ForConditionalGeneration (or the Tensorflow/Flax variant), which includes the language modeling head on top of the decoder.
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [5]:
sample = train[0]; sample

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [6]:
# %%timeit -n 1 -r 1

train = train.map(lambda x: squad_tokenizer_prompt(tokenizer, x), batched=True, num_proc=8)

Parameter 'function'=<function <lambda> at 0x7fb4a786ac10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/11 [00:00<?, ?ba/s]

#1:   0%|          | 0/11 [00:00<?, ?ba/s]

#2:   0%|          | 0/11 [00:00<?, ?ba/s]

#3:   0%|          | 0/11 [00:00<?, ?ba/s]

#4:   0%|          | 0/11 [00:00<?, ?ba/s]

#5:   0%|          | 0/11 [00:00<?, ?ba/s]

#6:   0%|          | 0/11 [00:00<?, ?ba/s]

#7:   0%|          | 0/11 [00:00<?, ?ba/s]

In [7]:
# %%timeit -n 1 -r 1

train = train.map(lambda x: squad_tokenizer_answer(tokenizer, x), batched=True, num_proc=8)

#0:   0%|          | 0/11 [00:00<?, ?ba/s]

#1:   0%|          | 0/11 [00:00<?, ?ba/s]

#2:   0%|          | 0/11 [00:00<?, ?ba/s]

#3:   0%|          | 0/11 [00:00<?, ?ba/s]

#4:   0%|          | 0/11 [00:00<?, ?ba/s]

#5:   0%|          | 0/11 [00:00<?, ?ba/s]

#6:   0%|          | 0/11 [00:00<?, ?ba/s]

#7:   0%|          | 0/11 [00:00<?, ?ba/s]

In [8]:
train.remove_columns("answers")

Dataset({
    features: ['context', 'question', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 87599
})